In [ ]:
# Imports
import pickle
import json
import os
from datetime import datetime
import sys
sys.path.append('..')

from load_data import load_dataset
from recommendations.recommendation_utils import (
    ResourceMatcher, 
    prepare_geographic_resources,
    enhance_recommendations_with_location
)

In [ ]:
# Load Data
tech_survey_df = load_dataset('tech_survey')
mental_health_care_df = load_dataset('mental_health_care')
who_suicide_df = load_dataset('who_suicide')

print(f"Tech survey: {len(tech_survey_df)} responses")
print(f"Mental health care: {len(mental_health_care_df)} records")
print(f"WHO data: {len(who_suicide_df)} records")

# Display available features for matching
print(f"\nTech survey features: {list(tech_survey_df.columns)}")

In [ ]:
# Prepare geographic resource database
geographic_resources = prepare_geographic_resources(mental_health_care_df, who_suicide_df)

# Initialize ResourceMatcher
enhanced_matcher = ResourceMatcher(tech_survey_df, geographic_resources)

# Define features for matching
matching_features = ['treatment', 'Gender', 'Age', 'Country'] 
enhanced_matcher.fit(matching_features)

print(f"Enhanced ResourceMatcher created with:")
print(f"- Survey responses: {len(tech_survey_df)}")
print(f"- Geographic resources for: {len(geographic_resources)} regions") 
print(f"- Matching features: {matching_features}")

In [ ]:
# Test resource matching
# Sample user profiles for testing
test_users = [
    {'treatment': 'No', 'Gender': 'Female', 'Age': 28, 'Country': 'United States'},
    {'treatment': 'Yes', 'Gender': 'Male', 'Age': 35, 'Country': 'Canada'},
    {'treatment': 'Maybe', 'Gender': 'Non-binary', 'Age': 30, 'Country': 'United Kingdom'}
]

for i, user_profile in enumerate(test_users):
    print(f"\n{'='*50}")
    print(f"USER {i+1}: {user_profile}")
    print('='*50)
    
    # Get enhanced recommendations with location
    enhanced_recommendations = enhanced_matcher.match_resources(
        user_profile, 
        include_location=True
    )
    
    # Get similarity statistics for analysis
    similarity_stats = enhanced_matcher.get_similarity_stats(user_profile)
    
    print(f"Similarity Stats:")
    for stat, value in similarity_stats.items():
        print(f"  {stat}: {value:.3f}")
    
    print(f"\nRecommended Resources:")
    for j, rec in enumerate(enhanced_recommendations[:8], 1):  # Show top 8
        print(f"  {j}. {rec}")

In [ ]:
# Compare location-enhanced vs standard recommendations
sample_user = {'treatment': 'No', 'Gender': 'Female', 'Age': 28, 'Country': 'United States'}

# Standard recommendations (no location enhancement)
standard_recs = enhanced_matcher.match_resources(sample_user, include_location=False)

# Location-enhanced recommendations  
enhanced_recs = enhanced_matcher.match_resources(sample_user, include_location=True)

print("COMPARISON: Standard vs Location-Enhanced Recommendations")
print("="*60)
print(f"User Profile: {sample_user}")

print(f"\nStandard Recommendations ({len(standard_recs)}):")
for i, rec in enumerate(standard_recs, 1):
    print(f"  {i}. {rec}")

print(f"\nLocation-Enhanced Recommendations ({len(enhanced_recs)}):")
for i, rec in enumerate(enhanced_recs, 1):
    print(f"  {i}. {rec}")

print(f"\nAdditional resources from location enhancement: {len(enhanced_recs) - len(standard_recs)}")

In [ ]:
# Create output directories
os.makedirs('../models/saved_models', exist_ok=True)
os.makedirs('../outputs/results', exist_ok=True)

# Save the enhanced matcher
timestamp = datetime.now().strftime('%Y%m%d')
model_filename = f"../models/saved_models/enhanced_resource_matcher_{timestamp}.pkl"

with open(model_filename, 'wb') as f:
    pickle.dump(enhanced_matcher, f)

print(f"Enhanced ResourceMatcher saved to: {model_filename}")

# Save test results and geographic resources
test_results = {
    'timestamp': timestamp,
    'model_type': 'enhanced_resource_matcher',
    'features_used': matching_features,
    'survey_records': len(tech_survey_df),
    'geographic_regions': len(geographic_resources),
    'test_users': test_users,
    'geographic_resources': geographic_resources
}

results_filename = f"../outputs/results/enhanced_recommendations_{timestamp}.json"

with open(results_filename, 'w') as f:
    json.dump(test_results, f, indent=2, default=str)

print(f"Test results and geographic database saved to: {results_filename}")
print(f"\nEnhanced recommendation system complete with {len(geographic_resources)} geographic regions")